**Deep Learning Notebook 2**

Dataset: MNIST digits dataset available as part of Keras

Objectives:
1. Vanishing & exploding gradients - He initialization
2. Batch normalization
3. Optimizers
4. Regularization: Dropout

**0. First Steps**

In [ ]:
# 0.1 Import the required libraries, modules

import tensorflow as tf
from tensorflow import keras

In [ ]:
# 0.2 Check the version details

print("TF Version: ", tf.__version__)
print("Keras Version: ", keras.__version__)

TF Version:  2.3.0
Keras Version:  2.4.0


In [ ]:
# 0.3 Load the dataset

mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
# 0.4 Check data shape

X_train_full.shape

(60000, 28, 28)

In [ ]:
# 0.5 Normalize by dividing by 255

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

In [ ]:
# 0.6 Set class names

class_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

End of First Steps

**1. Model with He initialization, batch normalization & Adam optimizer**

In [ ]:
# 1.1 Define model

def defmodel_3(n_hidden = 2, n_neurons = 150, learning_rate = 0.015):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation = "relu", kernel_initializer = "he_normal"))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(10, activation="softmax"))
    model.compile(loss = "sparse_categorical_crossentropy",
                  optimizer = keras.optimizers.Adam(lr = learning_rate, beta_1 = 0.9, beta_2 = 0.999),
                  metrics = ["accuracy"])
    return model  

# Instantiate model

model_3 = keras.wrappers.scikit_learn.KerasClassifier(defmodel_3)

In [ ]:
# 1.2 Grid search

# Import the required libraries/modules

from scipy.stats import reciprocal
from sklearn.model_selection import GridSearchCV
import numpy as np

# Initialize parameters dictionary

param_dict = {
    "n_hidden": [2, 3],
    "n_neurons": [100, 150, 200],
    "learning_rate": [0.01, 0.015, 0.02]
}

grid_search_cv_model_3 = GridSearchCV(model_3, param_dict, cv = 3, n_jobs = -1)
grid_search_cv_model_3.fit(X_train, y_train, epochs = 30, 
                           validation_data=(X_valid, y_valid),
                           callbacks=[keras.callbacks.EarlyStopping(patience=10)])

# Fetch best parameter values

print("Best parameter values: ", grid_search_cv_model_3.best_params_)

# Fetch best score

print("Best accuracy score: ", grid_search_cv_model_3.best_score_)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2831 - accuracy: 0.9127 - val_loss: 0.1945 - val_accuracy: 0.9548
Epoch 2/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.1698 - accuracy: 0.9478 - val_loss: 0.1585 - val_accuracy: 0.9510
Epoch 3/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.1406 - accuracy: 0.9571 - val_loss: 0.1107 - val_accuracy: 0.9670
Epoch 4/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.1188 - accuracy: 0.9628 - val_loss: 0.1008 - val_accuracy: 0.9706
Epoch 5/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.1020 - accuracy: 0.9680 - val_loss: 0.0982 - val_accuracy: 0.9736
Epoch 6/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.0938 - accuracy: 0.9714 - val_loss: 0.0813 - val_accuracy: 0.9770
Epoch 7/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.0848 - accuracy: 0.9734 - val_loss: 0.0756 - val_accuracy:

In [ ]:
# 1.3 Training the model on the entire train split

model_3_final = grid_search_cv_model_3.best_estimator_.model

history_3 = model_3_final.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0327 - accuracy: 0.9891 - val_loss: 0.0950 - val_accuracy: 0.9808
Epoch 2/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0345 - accuracy: 0.9895 - val_loss: 0.0910 - val_accuracy: 0.9780
Epoch 3/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0294 - accuracy: 0.9905 - val_loss: 0.1214 - val_accuracy: 0.9808
Epoch 4/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0303 - accuracy: 0.9905 - val_loss: 0.1133 - val_accuracy: 0.9780
Epoch 5/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0313 - accuracy: 0.9900 - val_loss: 0.1099 - val_accuracy: 0.9780
Epoch 6/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0300 - accuracy: 0.9898 - val_loss: 0.0886 - val_accuracy: 0.9802
Epoch 7/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.0284 - accuracy: 0.9910 - val_loss: 0.1006 - val_accuracy:

In [ ]:
# 1.4 Evaluate model on test dataset

model_3_final.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.2547 - accuracy: 0.9814


[0.25465306639671326, 0.9814000129699707]

End of Model 1

**2. Model with Regularization**

In [ ]:
# 2.1 Define model

def defmodel_4(n_hidden = 2, n_neurons = 150, learning_rate = 0.015):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, 
                                     activation = "relu", 
                                     kernel_initializer = "he_normal"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dropout(rate = 0.05))
    model.add(keras.layers.Dense(10, activation="softmax"))
    model.compile(loss = "sparse_categorical_crossentropy",
                  optimizer = keras.optimizers.Adam(lr = learning_rate, beta_1 = 0.9, beta_2 = 0.999),
                  metrics = ["accuracy"])
    return model  

# Instantiate model

model_4 = keras.wrappers.scikit_learn.KerasClassifier(defmodel_4)

In [ ]:
# 2.2 Grid search

# Import the required libraries/modules

from scipy.stats import reciprocal
from sklearn.model_selection import GridSearchCV
import numpy as np

# Initialize parameters dictionary

param_dict = {
    "n_hidden": [2, 3],
    "n_neurons": [100, 150, 200],
    "learning_rate": [0.01, 0.015, 0.02]
}

grid_search_cv_model_4 = GridSearchCV(model_4, param_dict, cv = 3, n_jobs = -1)
grid_search_cv_model_4.fit(X_train, y_train, epochs = 10, 
                           validation_data=(X_valid, y_valid),
                           callbacks=[keras.callbacks.EarlyStopping(patience=10)])

# Fetch best parameter values

print("Best parameter values: ", grid_search_cv_model_4.best_params_)

# Fetch best score

print("Best accuracy score: ", grid_search_cv_model_4.best_score_)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3030 - accuracy: 0.9087 - val_loss: 0.1644 - val_accuracy: 0.9524
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1847 - accuracy: 0.9433 - val_loss: 0.1118 - val_accuracy: 0.9668
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1519 - accuracy: 0.9536 - val_loss: 0.1009 - val_accuracy: 0.9708
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1345 - accuracy: 0.9581 - val_loss: 0.1106 - val_accuracy: 0.9676
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1214 - accuracy: 0.9627 - val_loss: 0.0946 - val_accuracy: 0.9730
Epoch 6/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1078 - accuracy: 0.9666 - val_loss: 0.0854 - val_accuracy: 0.9762
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.0944 - accuracy: 0.9713 - val_loss: 0.1047 - val_accuracy

In [ ]:
# 2.3 Training the model on the entire train split

model_4_final = grid_search_cv_model_4.best_estimator_.model

history_4 = model_4_final.fit(X_train, y_train, epochs=30)

Epoch 1/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.0695 - accuracy: 0.9778
Epoch 2/30
1719/1719 [==============================] - 8s 5ms/step - loss: 0.0670 - accuracy: 0.9789
Epoch 3/30
1719/1719 [==============================] - 8s 5ms/step - loss: 0.0655 - accuracy: 0.9795
Epoch 4/30
1719/1719 [==============================] - 8s 5ms/step - loss: 0.0603 - accuracy: 0.9812
Epoch 5/30
1719/1719 [==============================] - 8s 5ms/step - loss: 0.0582 - accuracy: 0.9817
Epoch 6/30
1719/1719 [==============================] - 8s 5ms/step - loss: 0.0543 - accuracy: 0.9830
Epoch 7/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.0580 - accuracy: 0.9819
Epoch 8/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.0525 - accuracy: 0.9837
Epoch 9/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.0518 - accuracy: 0.9835
Epoch 10/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.047

In [ ]:
# 2.4 Evaluate model on test dataset

model_4_final.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0992 - accuracy: 0.9783


[0.09918012470006943, 0.9782999753952026]